In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from tabulate import tabulate  # For formatted output

# Load dataset
file_path = "skinpro.csv"
df = pd.read_csv(file_path)

# Extract product type from product name
def get_product_type(name):
    name = name.lower()
    if "serum" in name:
        return "serum"
    elif "mask" in name:
        return "mask"
    elif "moisturizer" in name or "cream" in name:
        return "moisturizer"
    elif "face wash" in name or "cleanser" in name:
        return "face wash"
    elif "toner" in name:
        return "toner"
    elif "sunscreen" in name:
        return "sunscreen"
    else:
        return "other"

df["Product_Type"] = df["Product"].apply(get_product_type)
df['combined_text'] = df['Product_Type'] + " " + df['Skin type'].fillna('') + " " + df['Concern'].fillna('')

# Load Sentence-BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Compute embeddings for all products
product_embeddings = model.encode(df['combined_text'].tolist(), convert_to_tensor=True)

def recommend_products(user_query, top_n=10):
    # Compute user embedding
    user_embedding = model.encode([user_query], convert_to_tensor=True)

    # Compute cosine similarity
    similarities = cosine_similarity(user_embedding.cpu().numpy(), product_embeddings.cpu().numpy())[0]

    # Get top N recommendations
    top_indices = similarities.argsort()[-top_n:][::-1]
    recommended_products = df.iloc[top_indices][['Product', 'Skin type', 'Concern', 'product_url']]

    return recommended_products, similarities[top_indices]

def truncate_text(text, max_len=35):
    """Truncate text to a maximum length for cleaner display"""
    text = str(text)
    return text if len(text) <= max_len else text[:max_len-3] + "..."

def calculate_model_accuracy(queries):
    """Compute accuracy dynamically for given user queries."""
    mrr_total = 0
    precision_at_5_total = 0

    for query in queries:
        _, similarities = recommend_products(query, top_n=5)

        # Ensure similarities are numeric
        similarities = np.array(similarities, dtype=float)

        # Count relevant recommendations (similarity > 0.6)
        relevant_count = np.sum(similarities > 0.6)
        precision_at_5_total += relevant_count / 5

        # Compute reciprocal rank
        relevant_indices = np.where(similarities > 0.6)[0]
        reciprocal_rank = 1 / (relevant_indices[0] + 1) if len(relevant_indices) > 0 else 0
        mrr_total += reciprocal_rank

    mrr_score = mrr_total / len(queries)
    precision_at_5_score = precision_at_5_total / len(queries)

    return mrr_score, precision_at_5_score

# Get user input (support multiple concerns)
user_input = input("Enter your skin concerns (comma-separated, e.g., 'best sunscreen for dry skin, serum for acne'): ")
user_queries = [query.strip() for query in user_input.split(",")]

# Get recommendations for each query
for query in user_queries:
    recommendations, _ = recommend_products(query)

    # Truncate long columns for better display
    display_df = recommendations.copy()
    for col in ['Product', 'Skin type', 'Concern', 'product_url']:
        display_df[col] = display_df[col].apply(lambda x: truncate_text(x, max_len=35))

    print(f"\n🔹 **Top Recommendations for:** '{query}'\n")
    print(tabulate(display_df, headers="keys", tablefmt="fancy_grid", showindex=False))

# Compute & print overall model accuracy dynamically
mrr, precision_at_5 = calculate_model_accuracy(user_queries)
print(f"\n🔍 **Model Performance (Based on Your Queries):**")
print(f"📌 Mean Reciprocal Rank (MRR): {mrr:.3f}")
print(f"📌 Precision@5: {precision_at_5:.3f}")




/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Enter your skin concerns (comma-separated, e.g., 'best sunscreen for dry skin, serum for acne'): serum for wrinkles dry skin

🔹 **Top Recommendations for:** 'serum for wrinkles dry skin'

╒═════════════════════════════════════╤══════════════════════╤═════════════════════════╤═════════════════════════════════════╕
│ Product                             │ Skin type            │ Concern                 │ product_url                         │
╞═════════════════════════════════════╪══════════════════════╪═════════════════════════╪═════════════════════════════════════╡
│ Juicy Chemistry Australian Sanda... │ Dry                  │ Skin soothing           │ https://juicychemistry.com/produ... │
├─────────────────────────────────────┼──────────────────────┼─────────────────────────┼─────────────────────────────────────┤
│ Malazia HYDRA VEIL GEL MOISTURIZER  │ Dry                  │ Acne                    │ https://theformularx.com/product... │
├─────────────────────────────────────┼───────────